In [124]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import os
import glob
from time import sleep

In [35]:
# Importing cross currency sheet
cryptos = pd.read_csv('crosspairs.csv')

for idx, row in cryptos.iterrows():
    symbol = row['Symbol']
    exchanges = row.dropna().index.values[2:]
    
exchanges

array(['Cryptopia', 'Binance', 'Bittrex'], dtype=object)

In [121]:
# Pull from Cryptopia
def pulltopia(symbol):
    # Fields:
    # AskPrice BaseVolume BidPrice BuyBaseVolume BuyVolume Change
    # Close High Label LastPrice Low Open SellBaseVolume 
    # SellVolume TradePairId Volume
    
    topiasite = 'https://www.cryptopia.co.nz/api/GetMarket/' + symbol + '_BTC/0.001'
    response = requests.get(topiasite)
    json_data = json.loads(response.text)
    return pd.DataFrame(json_data['Data'], index = [0])

# Pull from kucoin
def pullkucoin(symbol):
    kubuy = 'https://api.kucoin.com/v1/' + symbol + '-BTC/open/orders-buy/'
    kusell = 'https://api.kucoin.com/v1/' + symbol + '-BTC/open/orders-sell/'
    respone = requests.get(kubuy)
    json_data_buy = json.loads(respone.text)
    response = requests.get(kusell)
    json_data_sell = json.loads(response.text)
    return pd.DataFrame({'BidPrice':[x[0] for x in json_data_buy['data']],
                         'BuyBaseVolume': [x[1] for x in json_data_buy['data']],
                         'AskPrice': [x[0] for x in json_data_sell['data']],
                         'SellBaseVolume': [x[1] for x in json_data_sell['data']]},
                        index = range(len(json_data_buy['data'])))

# Pull from Bittrex
def trexpull(symbol):
    trexsite = 'https://bittrex.com/api/v1.1/public/getorderbook?market=BTC-' + symbol + '&type=both'
    response = requests.get(trexsite)
    json_data = json.loads(response.text)
    return pd.concat([pd.DataFrame({'BidPrice': [x['Rate'] for x in json_data['result']['buy']],
                                    'BuyBaseVolume': [x['Quantity'] for x in json_data['result']['buy']]},
                                   index = range(len(json_data['result']['buy']))),
                      pd.DataFrame({'AskPrice': [x['Rate'] for x in json_data['result']['sell']],
                                    'SellBaseVolume': [x['Quantity'] for x in json_data['result']['sell']]},
                                   index = range(len(json_data['result']['sell'])))],
                     axis = 1)

# Pull from Quoine
def quoinepull(id):
    # currencypair id
    # UBTCETH	75
    # UBTCBTC	74
    # BTCUSD	1
    # QASHBTC	52
    # ETHBTC	37
    quoinesite = 'https://api.quoine.com/products/' + str(id) + '/price_levels'
    response = requests.get(quoinesite)
    json_data = json.loads(response.text)
    return pd.DataFrame({'BidPrice': [x[0] for x in json_data['buy_price_levels']],
                         'BuyBaseVolume': [x[1] for x in json_data['buy_price_levels']],
                         'AskPrice': [x[0] for x in json_data['sell_price_levels']],
                         'SellBaseVolume': [x[1] for x in json_data['sell_price_levels']]},
                        index = range(len(json_data['buy_price_levels'])))

In [159]:
# Arbitrage from Kucoin and Trex

def kutrexarb(symbol):
    kutrex = pd.read_csv('kutrex.csv')
    kutrex = kutrex['Symbol'].values
    #symbol = kutrex[-2]
    ku = pullkucoin(symbol)
    ku.columns = ['ku_' + colname for colname in ku]
    trex = trexpull(symbol)[:6]
    trex.columns = ['trex_' + colname for colname in trex]
    
    return pd.concat([ku,trex],axis = 1)

In [168]:
pulltopia('ETH')

,AskPrice,BaseVolume,BidPrice,BuyBaseVolume,BuyVolume,Change,Close,High,Label,LastPrice,Low,Open,SellBaseVolume,SellVolume,TradePairId,Volume
0,0.08701,593.960437,0.087,51.034507,3.322681e+07,-2.48,0.087,0.09135,ETH/BTC,0.087,0.075,0.089211,1.593835e+07,687.111513,5203,6767.864649


In [166]:
kutrexarb('ETH')

,ku_AskPrice,ku_BidPrice,ku_BuyBaseVolume,ku_SellBaseVolume,trex_BidPrice,trex_BuyBaseVolume,trex_AskPrice,trex_SellBaseVolume
0,0.087300,0.087000,0.328628,2.734687,0.087215,0.731894,0.087216,11.445439
1,0.087489,0.086900,1.586732,0.332845,0.086887,10.033816,0.087217,0.244686
2,0.087492,0.086803,0.207780,0.000425,0.086887,1.031642,0.087218,0.739915
3,0.087492,0.086777,0.092334,0.024121,0.086810,1.000000,0.087220,2.365684
4,0.087600,0.086734,0.172943,0.203746,0.086800,5.857140,0.087220,0.250000
5,0.088200,0.086732,0.345894,0.005000,0.086800,11.520701,0.087239,0.022618


In [171]:
quoinepull(37)[:6]

,AskPrice,BidPrice,BuyBaseVolume,SellBaseVolume
0,0.08330000,0.08000001,0.04000000,0.89000000
1,0.08355900,0.08000000,2.50000000,0.91000000
2,0.08380000,0.07861000,3.60080000,0.91000000
3,0.08398999,0.07860000,1.23601106,0.62068000
4,0.08400000,0.07827002,1.30000000,1.84500000
5,0.08599000,0.07727000,0.40000000,55.00000000
